In [14]:
import pybullet as p
import pybullet_data
import argparse
import numpy as np
import os
import os.path as osp
import joblib
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
import sys
from contextlib import contextmanager
from multiprocessing import Pool
import time
import plotly
import plotly.graph_objs as go
import json
import trimesh

In [289]:
# file_name = "data/parkour/motion/parkour-forward-roll.json"
file_name = "data/parkour/all_traj.json"
# file_name = "data/parkour/try_poses.json"
with open(file_name) as file:
    motion = json.load(file)

mesh_file_name = "data/parkour/object/Parkour_Speed Vault(L)/obj.obj"
mesh = trimesh.load(mesh_file_name)
vertices = mesh.vertices
vertices = mesh.vertices / 100

In [350]:
pelvis = np.array(motion["CC_Base_Pelvis"])

l_upperarm = np.array(motion["CC_Base_L_Upperarm"])
l_forearm = np.array(motion["CC_Base_L_Forearm"])
l_hand = np.array(motion["CC_Base_L_Hand"])

r_upperarm = np.array(motion["CC_Base_R_Upperarm"])
r_forearm = np.array(motion["CC_Base_R_Forearm"])
r_hand = np.array(motion["CC_Base_R_Hand"])

l_thigh = np.array(motion["CC_Base_L_Thigh"])
l_calf = np.array(motion["CC_Base_L_Calf"])
l_foot = np.array(motion["CC_Base_L_Foot"])
l_toe = np.array(motion["CC_Base_L_ToeBase"])

r_thigh = np.array(motion["CC_Base_R_Thigh"])
r_calf = np.array(motion["CC_Base_R_Calf"])
r_foot = np.array(motion["CC_Base_R_Foot"])
r_toe = np.array(motion["CC_Base_R_ToeBase"])

skeleton = np.array(
    [
        pelvis,
        l_thigh,
        l_calf,
        l_foot,
        l_toe,
        l_upperarm,
        l_forearm,
        l_hand,
        r_thigh,
        r_calf,
        r_foot,
        r_toe,
        r_upperarm,
        r_forearm,
        r_hand,
    ]
)

skeleton = skeleton.transpose(1,0,2) / 100

In [351]:
skeleton.shape

(10001, 15, 3)

In [275]:
new_clip = []
for i in range(len(pelvis) - 1):
    if np.linalg.norm(skeleton[i, 0] - skeleton[i+1, 0]) > 0.7:
        new_clip.append(i)
print(len(new_clip))
last_frame = 9806

44


In [318]:
new_clip

[167,
 330,
 663,
 949,
 1273,
 1516,
 1774,
 2091,
 2300,
 2431,
 2558,
 2746,
 2915,
 3352,
 3626,
 3799,
 3985,
 4182,
 4367,
 4552,
 4797,
 4980,
 5195,
 5345,
 5525,
 5665,
 5842,
 6006,
 6213,
 6368,
 6568,
 6773,
 7074,
 7332,
 7492,
 7701,
 8101,
 8358,
 8614,
 8884,
 9114,
 9349,
 9507,
 9675]

In [334]:
motion_names = ['Parkour_One-handed Rotation',
'Parkour_Swan Dive',
'Parkour_Underbar Flip',
'Parkour_Palmspin Jump',
'Parkour_Lache Jump',
'Parkour_Reverse-grip Rotation',
'Parkour_Reverse Landing',
'Parkour_Forward Roll & Side Flip',
'Parkour_Climb up(H)',
'Parkour_Drop(H)',
'Parkour_Climb up(M)',
'Parkour_Drop(M) Forward Roll',
'Parkour_Drop(M) Side Flip Jump',
'Parkour_Climb & Side Flip Jump',
'Parkour_Slideway',
'Parkour_Reverse Vault',
'Parkour_Forward Handspring',
'Parkour_Roundoff',
'Parkour_Flare',
'Parkour_Side Flip Jump',
'Parkour_Aerial',
'Parkour_360 Backflip',
'Parkour_Tic Tac Backflip',
'Parkour_Cat Vault',
'Parkour_Kash Vault',
'Parkour_Speed Vault(L)',
'Parkour_Speed Vault(R)',
'Parkour_Back Roll Axel Jump',
'Parkour_Webster',
'Parkour_Side Flip',
'Parkour_Hands Cork',
'Parkour_Cork & Roll',
'Parkour_Kong Combo',
'Parkour_Wall Hop',
'Parkour_Drop Running',
'Parkour_Sit Drop',
'Parkour_Precision Jump',
'Parkour_Flapping Jump & Roll',
'Parkour_Turn Vault',
'Parkour_Back Flip',
'Parkour_Roll & Flip',
'Parkour_Aerial x3',
'Parkour_Slow Down',
'Parkour_Forward Roll',
'Parkour_Hook Slide']

In [335]:
string_count = {}

for item in motion_names:
    if item in string_count:
        string_count[item] += 1
    else:
        string_count[item] = 1

for string, count in string_count.items():
    if count > 1:
        print(f"The string '{string}' appears {count} times in the list.")

In [336]:
a = set(motion_names)
len(a)

45

In [337]:
for motion_name in motion_names:
    if not osp.isfile(f"data/parkour/motions/{motion_name}.npy"):
        print(motion_name)

Parkour_Webster


In [355]:
for i in range(len(new_clip) + 1):
    if i == 0:
        start = 0
    else:
        start = new_clip[i-1] + 1
    if i == len(new_clip):
        end = last_frame
    else:
        end = new_clip[i]
    print(start, end)
    skeleton_clip = skeleton[start:end]
    print(skeleton_clip.shape)
    np.save(f"data/parkour/motions/{motion_names[i]}.npy", skeleton_clip[:,:,[2,0,1]])

0
0 167
(167, 15, 3)
1
168 330
(162, 15, 3)
2
331 663
(332, 15, 3)
3
664 949
(285, 15, 3)
4
950 1273
(323, 15, 3)
5
1274 1516
(242, 15, 3)
6
1517 1774
(257, 15, 3)
7
1775 2091
(316, 15, 3)
8
2092 2300
(208, 15, 3)
9
2301 2431
(130, 15, 3)
10
2432 2558
(126, 15, 3)
11
2559 2746
(187, 15, 3)
12
2747 2915
(168, 15, 3)
13
2916 3352
(436, 15, 3)
14
3353 3626
(273, 15, 3)
15
3627 3799
(172, 15, 3)
16
3800 3985
(185, 15, 3)
17
3986 4182
(196, 15, 3)
18
4183 4367
(184, 15, 3)
19
4368 4552
(184, 15, 3)
20
4553 4797
(244, 15, 3)
21
4798 4980
(182, 15, 3)
22
4981 5195
(214, 15, 3)
23
5196 5345
(149, 15, 3)
24
5346 5525
(179, 15, 3)
25
5526 5665
(139, 15, 3)
26
5666 5842
(176, 15, 3)
27
5843 6006
(163, 15, 3)
28
6007 6213
(206, 15, 3)
29
6214 6368
(154, 15, 3)
30
6369 6568
(199, 15, 3)
31
6569 6773
(204, 15, 3)
32
6774 7074
(300, 15, 3)
33
7075 7332
(257, 15, 3)
34
7333 7492
(159, 15, 3)
35
7493 7701
(208, 15, 3)
36
7702 8101
(399, 15, 3)
37
8102 8358
(256, 15, 3)
38
8359 8614
(255, 15, 3)
39
8615

In [354]:
skeleton[start:end].shape

(130, 15, 3)

In [339]:
a = trimesh.load('data/parkour/object/Parkour_Lache Jump/obj.obj')
len(a.geometry)

2

In [342]:
folders = os.listdir('data/parkour/object')
for folder in folders:
    if not osp.isfile(f"data/parkour/objects/{folder}.obj"):
        print(folder)

In [359]:
for motion_name in motion_names:
    if osp.isfile(f"data/parkour/object/{motion_name}/obj.obj"):
        mesh = trimesh.load(f"data/parkour/object/{motion_name}/obj.obj")
        if isinstance(mesh, trimesh.Scene):
            scene = trimesh.Scene()
            for i, geometry in enumerate(mesh.geometry.values()):
                vertices = geometry.vertices.copy()
                vertices = vertices[:, [2,0,1]] / 100
                scene.add_geometry(trimesh.Trimesh(vertices, geometry.faces))
            scene.export(f"data/parkour/objects/{motion_name}.obj")
        else:
            vertices = mesh.vertices.copy()
            vertices = vertices[:, [2,0,1]] / 100
            new_mesh = trimesh.Trimesh(vertices, mesh.faces)
            new_mesh.export(f"data/parkour/objects/{motion_name}.obj")

In [360]:
skeleton = np.load(f"data/parkour/motions/{'Parkour_Speed Vault(L)'}.npy")
obstacle = trimesh.load(f"data/parkour/objects/{'Parkour_Speed Vault(L)'}.obj")
vertices = obstacle.vertices

In [361]:
skeleton.shape

(139, 15, 3)

In [362]:
connection = [[0,1], [1,2], [2,3], [3,4], [0,5], [5,6], [6,7], [0,8], [8,9], [9,10], [10,11], [0,12], [12,13], [13,14]]

data = []

frame_idx = 0
joints_1 = go.Scatter3d(
    x=skeleton[frame_idx, :, 0],  
    y=skeleton[frame_idx, :, 1],  
    z=skeleton[frame_idx, :, 2], 
    mode='markers',
    marker={
        'size': 6,
        'opacity': 0.8,
    }
)

for c in connection:
    line_trace = go.Scatter3d(
        x=skeleton[frame_idx, c, 0],
        y=skeleton[frame_idx, c, 1],
        z=skeleton[frame_idx, c, 2],
        mode='lines',
        line=dict(color='red', width=4)
    )
    data.append(line_trace)

frame_idx = 60
joints_2 = go.Scatter3d(
    x=skeleton[frame_idx, :, 0],  
    y=skeleton[frame_idx, :, 1],  
    z=skeleton[frame_idx, :, 2], 
    mode='markers',
    marker={
        'size': 6,
        'opacity': 0.8,
    }
)

for c in connection:
    line_trace = go.Scatter3d(
        x=skeleton[frame_idx, c, 0],
        y=skeleton[frame_idx, c, 1],
        z=skeleton[frame_idx, c, 2],
        mode='lines',
        line=dict(color='red', width=4)
    )
    data.append(line_trace)

layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        scene = dict(
            # xaxis = dict(title="x", range = [-2,2]),
            # yaxis = dict(title="y", range = [-1,1]),
            # zaxis = dict(title="z", range = [0,2]),
            aspectratio=dict(x=1, y=1, z=1)
    ))

obstacle = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], color='lightpink', opacity=0.50)


# fig = go.Figure(data=data + [joints_1, joints_2, obstacle], layout=layout)
fig = go.Figure(data= data + [obstacle], layout=layout)
# plotly.offline.iplot(fig)
fig.show()

In [217]:
robot_urdf = "/home/jianrenw/mocap/robots/adam_lite/urdf/adam_lite_pybullet.urdf"

physicsClient = p.connect(p.DIRECT)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])

humanoid = p.loadURDF(robot_urdf, robot_start_pos, robot_start_orientation)

time_step = 1.0 / 240.0
p.setTimeStep(time_step)

num_joints = p.getNumJoints(humanoid)

joint_names = []
joint_lower_limits = []
joint_upper_limits = []

# Iterate through the joints and print their names
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(humanoid, joint_index)
    joint_name = joint_info[1].decode(
        "utf-8"
    )  # Decode the byte string to get the joint name
    joint_lower_limit = joint_info[8]  # Lower joint limit
    joint_upper_limit = joint_info[9]  # Upper joint limit
    joint_names.append(joint_name)
    joint_lower_limits.append(joint_lower_limit)
    joint_upper_limits.append(joint_upper_limit)

joint_lower_limits = np.array(joint_lower_limits)
joint_upper_limits = np.array(joint_upper_limits)

p.disconnect()

In [153]:
# upper body
head = (l_upperarm + r_upperarm) / 2
upper_z = (head - pelvis) / np.linalg.norm(head - pelvis, axis=1, keepdims=True)
upper_x = np.cross(pelvis - l_upperarm, pelvis - r_upperarm)
upper_x = upper_x / np.linalg.norm(upper_x, axis=1, keepdims=True)
upper_y = np.cross(upper_z, upper_x)
upper_rot = np.stack([upper_x, upper_y, upper_z], axis=2)

# lower body
crotch = (l_thigh + r_thigh) / 2
lower_z = - (crotch - pelvis) / np.linalg.norm(crotch - pelvis, axis=1, keepdims=True)
lower_x = np.cross(pelvis - r_thigh, pelvis - l_thigh)
lower_x = lower_x / np.linalg.norm(lower_x, axis=1, keepdims=True)
lower_y = np.cross(lower_z, lower_x)
lower_rot = np.stack([lower_x, lower_y, lower_z], axis=2)

# left arm
l_elbow_rot_z = l_forearm - l_hand
l_elbow_rot_z = l_elbow_rot_z / np.linalg.norm(
    l_elbow_rot_z, axis=1, keepdims=True
)
l_elbow_rot_y_2 = l_upperarm - l_forearm
l_elbow_rot_y_2 = l_elbow_rot_y_2 / np.linalg.norm(
    l_elbow_rot_y_2, axis=1, keepdims=True
)
l_elbow_rot_y_2 = np.cross(l_elbow_rot_y_2, -l_elbow_rot_z)
l_elbow_rot_y_2 = l_elbow_rot_y_2 / np.linalg.norm(
    l_elbow_rot_y_2, axis=1, keepdims=True
)

l_elbow_rot_x = np.cross(l_elbow_rot_y_2, l_elbow_rot_z)
l_elbow_rot_x = l_elbow_rot_x / np.linalg.norm(
    l_elbow_rot_x, axis=1, keepdims=True
)

l_elbow_rot = np.stack([l_elbow_rot_x, l_elbow_rot_y_2, l_elbow_rot_z], axis=2)

l_upperarm_dir = l_forearm - l_upperarm
l_upperarm_dir = l_upperarm_dir / np.linalg.norm(
    l_upperarm_dir, axis=1, keepdims=True
)

# right arm
r_elbow_rot_z = r_forearm - r_hand
r_elbow_rot_z = r_elbow_rot_z / np.linalg.norm(
    r_elbow_rot_z, axis=1, keepdims=True
)
r_elbow_rot_y_2 = r_upperarm - r_forearm
r_elbow_rot_y_2 = r_elbow_rot_y_2 / np.linalg.norm(
    r_elbow_rot_y_2, axis=1, keepdims=True
)   
r_elbow_rot_y_2 = np.cross(r_elbow_rot_y_2, -r_elbow_rot_z)
r_elbow_rot_y_2 = r_elbow_rot_y_2 / np.linalg.norm(
    r_elbow_rot_y_2, axis=1, keepdims=True
)

r_elbow_rot_x = np.cross(r_elbow_rot_y_2, r_elbow_rot_z)
r_elbow_rot_x = r_elbow_rot_x / np.linalg.norm(
    r_elbow_rot_x, axis=1, keepdims=True
)
r_elbow_rot = np.stack([r_elbow_rot_x, r_elbow_rot_y_2, r_elbow_rot_z], axis=2)

r_upperarm_dir = r_forearm - r_upperarm
r_upperarm_dir = r_upperarm_dir / np.linalg.norm(
    r_upperarm_dir, axis=1, keepdims=True
)

# left knee
l_calf_dir = l_calf - l_foot
l_calf_dir = l_calf_dir / np.linalg.norm(
    l_calf_dir, axis=1, keepdims=True
)
l_foot_dir = l_toe - l_foot
l_foot_dir = l_foot_dir / np.linalg.norm(
    l_foot_dir, axis=1, keepdims=True
)
l_knee_rot_y = np.cross(l_calf_dir, l_foot_dir)
l_knee_rot_y = l_knee_rot_y / np.linalg.norm(
    l_knee_rot_y, axis=1, keepdims=True
)
l_knee_rot_x = np.cross(l_knee_rot_y, l_calf_dir)
l_knee_rot_x = l_knee_rot_x / np.linalg.norm(
    l_knee_rot_x, axis=1, keepdims=True
)
l_knee_rot = np.stack([l_knee_rot_x, l_knee_rot_y, l_calf_dir], axis=2)

l_thigh_dir = l_calf - l_thigh
l_thigh_dir = l_thigh_dir / np.linalg.norm(l_thigh_dir, axis=1, keepdims=True)

# left ankle angle
cos = np.einsum('ij,ij->i', l_calf_dir, l_foot_dir) / (np.linalg.norm(l_calf_dir, axis=1) * np.linalg.norm(l_foot_dir, axis=1))
l_foot_angle = np.arccos(cos) - np.arctan(0.047/0.160) - np.pi/2

# right knee
r_calf_dir = r_calf - r_foot
r_calf_dir = r_calf_dir / np.linalg.norm(
    r_calf_dir, axis=1, keepdims=True
)
r_foot_dir = r_toe - r_foot
r_foot_dir = r_foot_dir / np.linalg.norm(
    r_foot_dir, axis=1, keepdims=True
)
r_knee_rot_y = np.cross(r_calf_dir, r_foot_dir)
r_knee_rot_y = r_knee_rot_y / np.linalg.norm(
    r_knee_rot_y, axis=1, keepdims=True
)
r_knee_rot_x = np.cross(r_knee_rot_y, r_calf_dir)
r_knee_rot_x = r_knee_rot_x / np.linalg.norm(
    r_knee_rot_x, axis=1, keepdims=True
)
r_knee_rot = np.stack([r_knee_rot_x, r_knee_rot_y, r_calf_dir], axis=2)

r_thigh_dir = r_calf - r_thigh
r_thigh_dir = r_thigh_dir / np.linalg.norm(r_thigh_dir, axis=1, keepdims=True)

# right ankle angle
cos = np.einsum('ij,ij->i', r_calf_dir, r_foot_dir) / (np.linalg.norm(r_calf_dir, axis=1) * np.linalg.norm(r_foot_dir, axis=1))
r_foot_angle = np.arccos(cos) - np.arctan(0.047/0.160) - np.pi/2


# to pybullet
upper_rot = R.from_matrix(upper_rot)
lower_rot = R.from_matrix(lower_rot)
l_knee_rot = R.from_matrix(l_knee_rot)
r_knee_rot = R.from_matrix(r_knee_rot)
l_elbow_rot = R.from_matrix(l_elbow_rot)
r_elbow_rot = R.from_matrix(r_elbow_rot)

# waist angle
R_u_l = np.einsum('ijk,ikl->ijl', np.transpose(lower_rot.as_matrix(), (0, 2, 1)), upper_rot.as_matrix())
R_u_l = R.from_matrix(R_u_l)
waist_angles = R_u_l.as_euler('xyz', degrees=False)


amass_data = {'num_frames': num_frames,
            'pelvis': pelvis,
            'upper_rot': upper_rot,
            'lower_rot': lower_rot,
            'l_elbow_rot': l_elbow_rot,
            'r_elbow_rot': r_elbow_rot,
            'l_knee_rot': l_knee_rot,
            'r_knee_rot': r_knee_rot,
            'l_upperarm_dir': l_upperarm_dir,
            'r_upperarm_dir': r_upperarm_dir,
            'l_thigh_dir': l_thigh_dir,
            'r_thigh_dir': r_thigh_dir,
            'waist_angles': waist_angles,
            'l_foot_angle': l_foot_angle,
            'r_foot_angle': r_foot_angle,
            'l_foot_dir': l_foot_dir,
            'r_foot_dir': r_foot_dir,
}

In [154]:
pelvis

array([[-5.74045463e-03,  1.28846345e-03,  9.60034517e-01],
       [-2.77447365e-03,  2.34741778e-04,  9.58093652e-01],
       [ 1.80230717e-03, -2.54084745e-04,  9.55832565e-01],
       [ 6.42628428e-03, -6.31187842e-04,  9.53044016e-01],
       [ 1.16708881e-02, -1.28798307e-03,  9.50088038e-01],
       [ 1.56740669e-02, -2.26705319e-03,  9.46551714e-01],
       [ 2.12268737e-02, -2.84220665e-03,  9.43369371e-01],
       [ 2.54838165e-02, -3.83225248e-03,  9.40029460e-01],
       [ 2.91159793e-02, -5.03078548e-03,  9.36506722e-01],
       [ 3.19440942e-02, -6.09174865e-03,  9.32995192e-01],
       [ 3.27428027e-02, -7.59569231e-03,  9.28732596e-01],
       [ 3.36558434e-02, -9.10993688e-03,  9.23557837e-01],
       [ 3.50645334e-02, -9.98969760e-03,  9.16852231e-01],
       [ 3.57798348e-02, -1.04495125e-02,  9.07908460e-01],
       [ 3.69682203e-02, -1.02735526e-02,  8.97125556e-01],
       [ 3.91168719e-02, -9.15118005e-03,  8.84082775e-01],
       [ 4.27881288e-02, -8.67353149e-03

In [155]:
physicsClient = p.connect(p.GUI)  # non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])
humanoid = p.loadURDF(robot_urdf, robot_start_pos, robot_start_orientation)


num_frames = amass_data['num_frames']
pelvis = amass_data['pelvis']
upper_rot = amass_data['upper_rot']
lower_rot = amass_data['lower_rot']
l_elbow_rot = amass_data['l_elbow_rot']
r_elbow_rot = amass_data['r_elbow_rot']
l_knee_rot = amass_data['l_knee_rot']
r_knee_rot = amass_data['r_knee_rot']
l_upperarm_dir = amass_data['l_upperarm_dir']
r_upperarm_dir = amass_data['r_upperarm_dir']
l_thigh_dir = amass_data['l_thigh_dir']
r_thigh_dir = amass_data['r_thigh_dir']
waist_angles = amass_data['waist_angles']
l_foot_angle = amass_data['l_foot_angle']
r_foot_angle = amass_data['r_foot_angle']
l_foot_dir = amass_data['l_foot_dir']
r_foot_dir = amass_data['r_foot_dir']

upperarm_len = 0.2648365892539233
thigh_len = 0.4252746432372357
toe_len = 0.167
heel_len = 0.079

toe_R = R.from_euler('y', np.arctan(0.047/0.160)).as_matrix()
heel_R = R.from_euler('y', np.pi / 2 + np.arctan(0.047/0.064)).as_matrix()

num_joints = p.getNumJoints(humanoid)

joint_names = []

pelvis_copy = pelvis.copy()

# Iterate through the joints and print their names
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(humanoid, joint_index)
    joint_name = joint_info[1].decode(
        "utf-8"
    )  # Decode the byte string to get the joint name
    joint_names.append(joint_name)

joint_poses = []
jointDamping = [0.1] * num_joints
jointDamping[12] = 100
jointDamping[13] = 100
jointDamping[14] = 100
restPoses = [0.0] * num_joints
init_pose = [0.0] * num_joints

min_foot_z = np.inf

for i in range(num_frames):

    init_pose[12] = waist_angles[i,0]
    init_pose[13] = waist_angles[i,1]
    init_pose[14] = waist_angles[i,2]
    restPoses[12] = waist_angles[i,0]
    restPoses[13] = waist_angles[i,1]
    restPoses[14] = waist_angles[i,2]

    p.resetBasePositionAndOrientation(
        humanoid, (pelvis[i]).tolist(), lower_rot[i].as_quat().tolist()
    )
    for joint_index, angle in enumerate(init_pose):
        p.resetJointState(humanoid, joint_index, angle)

    l_shoulder_pos, _ = p.getLinkState(
        humanoid, joint_names.index("shoulderRoll_Left")
    )[4:6]
    r_shoulder_pos, _ = p.getLinkState(
        humanoid, joint_names.index("shoulderRoll_Right")
    )[4:6]
    l_hip_pos, _ = p.getLinkState(
        humanoid, joint_names.index("hipRoll_Left")
    )[4:6]
    r_hip_pos, _ = p.getLinkState(
        humanoid, joint_names.index("hipRoll_Right")
    )[4:6]

    # IK for l hand
    l_elbow_pos = np.array(l_shoulder_pos) + l_upperarm_dir[i] * upperarm_len

    # IK for r hand
    r_elbow_pos = np.array(r_shoulder_pos) + r_upperarm_dir[i] * upperarm_len

    # IK for l foot
    l_knee_pos = np.array(l_hip_pos) + l_thigh_dir[i] * thigh_len

    # IK for r foot
    r_knee_pos = np.array(r_hip_pos) + r_thigh_dir[i] * thigh_len

    # # Calculate the IK solution
    ik_solution_rh = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("elbow_Right"),
        r_elbow_pos,
        targetOrientation=r_elbow_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed,
        restPoses=restPoses,  # Initial joint angles
    )

    ik_solution_lh = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("elbow_Left"),
        l_elbow_pos,
        targetOrientation=l_elbow_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed
        restPoses=restPoses,  # Initial joint anglesss
    )

    ik_solution_rf = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("kneePitch_Right"),
        r_knee_pos,
        targetOrientation=r_knee_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed
        restPoses=restPoses,  # Initial joint angles
    )

    ik_solution_lf = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("kneePitch_Left"),
        l_knee_pos,
        targetOrientation=l_knee_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed
        restPoses=restPoses,  # Initial joint angles
    )

    # p.resetJointState(humanoid, joint_index, ik_solution)

    # adam
    # ik_solution = np.zeros(num_joints)
    # ik_solution[:4] = ik_solution_lf[:4]
    # ik_solution[4] = l_foot_angle[i]
    # ik_solution[6:10] = ik_solution_rf[6:10]
    # ik_solution[10] = r_foot_angle[i]
    # ik_solution[12] = waist_angles[i,0]
    # ik_solution[13] = waist_angles[i,1]
    # ik_solution[14] = waist_angles[i,2]
    # ik_solution[16:20] = ik_solution_lh[15:19]
    # ik_solution[24:28] = ik_solution_rh[19:23]

    # adam lite with wrist
    # ik_solution = np.zeros(num_joints)
    # ik_solution[:4] = ik_solution_lf[:4]
    # ik_solution[4] = l_foot_angle[i]
    # ik_solution[6:10] = ik_solution_rf[6:10]
    # ik_solution[10] = r_foot_angle[i]
    # ik_solution[12] = waist_angles[i,0]
    # ik_solution[13] = waist_angles[i,1]
    # ik_solution[14] = waist_angles[i,2]
    # ik_solution[16:20] = ik_solution_lh[15:19]
    # ik_solution[22:26] = ik_solution_rh[20:24]

    # adam lite
    ik_solution = np.zeros(num_joints)
    ik_solution[:4] = ik_solution_lf[:4]
    ik_solution[4] = l_foot_angle[i]
    ik_solution[6:10] = ik_solution_rf[6:10]
    ik_solution[10] = r_foot_angle[i]
    ik_solution[12] = waist_angles[i,0]
    ik_solution[13] = waist_angles[i,1]
    ik_solution[14] = waist_angles[i,2]
    ik_solution[16:20] = ik_solution_lh[15:19]
    ik_solution[22:26] = ik_solution_rh[19:23]

    restPoses = ik_solution
    init_pose = ik_solution

    for joint_index, angle in enumerate(init_pose):
        p.resetJointState(humanoid, joint_index, angle)

    l_ankle_pos, l_ankle_rot = p.getLinkState(
        humanoid, joint_names.index("anklePitch_Left")
    )[4:6]
    r_ankle_pos, r_ankle_rot = p.getLinkState(
        humanoid, joint_names.index("anklePitch_Right")
    )[4:6]

    l_rot_matrix = p.getMatrixFromQuaternion(l_ankle_rot)
    l_rot_matrix = np.array(l_rot_matrix).reshape(3, 3)
    l_toe_pose = np.dot(l_rot_matrix@toe_R, [toe_len, 0, 0]) + l_ankle_pos
    l_heel_pose = np.dot(l_rot_matrix@heel_R, [heel_len, 0, 0]) + l_ankle_pos

    r_rot_matrix = p.getMatrixFromQuaternion(r_ankle_rot)
    r_rot_matrix = np.array(r_rot_matrix).reshape(3, 3)
    r_toe_pose = np.dot(r_rot_matrix@toe_R, [toe_len, 0, 0]) + r_ankle_pos
    r_heel_pose = np.dot(r_rot_matrix@heel_R, [heel_len, 0, 0]) + r_ankle_pos

    if i < 30:
        current_min = np.min([l_toe_pose[2], r_toe_pose[2], l_heel_pose[2], r_heel_pose[2]])
        if min_foot_z > current_min:
            min_foot_z = current_min

    pelvis_copy[i, 2] = pelvis[i,2]

    # Step the simulation
    joint_poses.append(ik_solution)

    time.sleep(0.1)

p.disconnect()

result = {
    'root_pos': pelvis_copy - min_foot_z,
    'root_rot': lower_rot.as_quat(),
    'joint_poses': np.array(joint_poses),
    'joint_names': joint_names
}

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 545.29.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 545.29.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
dest

In [156]:
result['real_frame_rate'] = 60

In [159]:
joblib.dump(result, "data/'Parkour/out/" + "{}_data.pt".format(''Parkour-forward-roll'))

['data/parkour/out/parkour-forward-roll_data.pt']

In [141]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
destroy semaphore
semaphore destroyed
Thread TERMINATED
destroy main semaphore
main semaphore destroyed


In [133]:
robot_urdf = "/home/jianrenw/Research/humanoid_'Parkour/retarget/h1_description/urdf/h1.urdf"

physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])

humanoid = p.loadURDF(robot_urdf, robot_start_pos, robot_start_orientation)

# Start the base position update loop
joint_poses = ik(humanoid)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 545.29.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 545.29.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,152]:

b3Printf: URDF file '/home/jianrenw/Research/humanoid_parkour/retarget/h1_description/urdf/h1

error: Cannot load URDF file.

ven = NVIDIA Corporation


## Save Video

In [ ]:
robot_urdf = "/home/jianrenw/Research/humanoid_'Parkour/retarget/h1_description/urdf/h1.urdf"

physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])

humanoid = p.loadURDF(robot_urdf, robot_start_pos, robot_start_orientation)

video_filename = "simulation_h1_video.mp4"
frame_width = 1280  # Adjust the frame width as needed
frame_height = 720  # Adjust the frame height as needed
frame_rate = 30  # Adjust the frame rate (frames per second) as needed


time.sleep(5)

p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, video_filename)

for i in range(frame_num):
    key_t = translation[i] / 100
    key_r = base_rot[i]
    p.resetBasePositionAndOrientation(humanoid, key_t[0].tolist(), key_r.tolist())
    for joint_index, angle in enumerate(joint_poses[i]):
        p.resetJointState(humanoid, joint_index, angle)
    p.stepSimulation()
    time.sleep(5.0 / frame_rate)


p.stopStateLogging(p.STATE_LOGGING_VIDEO_MP4)
p.disconnect()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA Graphics Device/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 520.61.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 520.61.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA Graphics Device/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagona

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed


In [10]:
np.array(joint_poses)

array([[ 0.16987518,  0.01599213, -0.34437726, ...,  0.06131537,
         0.        ,  0.        ],
       [ 0.19750815,  0.02771593, -0.55568496, ..., -0.09586451,
         0.        ,  0.        ],
       [ 0.19737979,  0.02682791, -0.61939508, ..., -0.09432242,
         0.        ,  0.        ],
       ...,
       [ 0.28426226, -0.16334513, -0.49797419, ...,  0.18871116,
         0.        ,  0.        ],
       [ 0.2775931 , -0.17256745, -0.45473287, ...,  0.20171518,
         0.        ,  0.        ],
       [ 0.27887362, -0.17523028, -0.43649568, ...,  0.20163231,
         0.        ,  0.        ]])